# Trabajo Práctico

Elegir un tema, persona, situación o evento y crear un sistema que:
- Obtenga los Tweets asociados con dicho tema (Hashtag y/o Palabras claves)
- Aplique algunas de las técnicas de PNL (Tokenización, POS, Lematización, Derivación, Polarización)
- Obtener algunos de los siguientes datos (¿Quíenes están involucrados?, ¿De qué se trata?, ¿En qué lugares ocurre la acción?, ¿Qué opinion o sentimientos expresa la gente?
- Generar resultados (Gráficos, tablas )

## Obtener datos

In [18]:
import os
import pandas as pd
import requests
from dotenv import load_dotenv

# Cargar valores del archivo .env en las variables de entorno
load_dotenv()
# Cargar valor del token a variable
bearer_token = os.environ.get("BEARER_TOKEN")

url = "https://api.twitter.com/2/tweets/search/recent"
headers = {
    "Authorization": f"Bearer {bearer_token}",
    "User-Agent":"v2FullArchiveSearchPython"
} 
params = {
    'query': f'#metaverse lang:en -is:retweet',
    'expansions': 'author_id',
    'max_results':50
}
# Función para obtener hasta 10 paginas del tema buscado.
def get_data(url,params):
    results = []

    for _ in range(5):
        response = requests.get(url, headers=headers, params=params)
        # Generar excepción si la respuesta no es exitosa
        if response.status_code != 200:
            raise Exception(response.status_code, response.text)
        data = response.json()['data']
        meta_data = dict(response.json())['meta']
        results.append(pd.json_normalize(data))
        if 'next_token' not in meta_data:
            break
        else:
            token = meta_data['next_token']
            params = {
                'query': f'#metaverse lang:en -is:retweet',
                'expansions': 'author_id',
                'next_token':token,
                'max_results':50
            }
    
    return pd.concat(results)

df = get_data(url,params)
df.to_csv('metaverse.csv')  

In [48]:
df


,Unnamed: 0,author_id,id,text
0,0,1389675288255205378,1454461866906894339,bull tpt price is cheap buy cheapsign up for gateio discountbabydogearmy metaverse paribu btcturk mexcglobal shibarmy babydogearmy feg fegtoken binance binancesmartchain bnb token bitexen dogecointothemoon dogezilla
1,1,1448338535342231553,1454461853929639943,im definitely not underexposed i have s invested thousands in soul because is perfect for the metaverse phantasma is a layer blockchain with builtin smartnft technology perfect for nftgames and blockchaingaming
2,2,1441702148257955843,1454461843993292804,ilove elemongame partner corsairgamenft playtoearn topmvb binancesmartchainmetaverse
3,3,1440648481970950152,1454461830772899843,members will get sapce token randomlymetaverse presale gamefi nft playtoearn pe
4,4,44817501,1454461819372724238,and if you wanna know abt metaverse in a beginner friendly wayread this thread
5,5,1441702148257955843,1454461817120460802,ilove elemongame partner corsairgamenft playtoearn topmvb binancesmartchainmetaverse
6,6,1430501884192858112,1454461799210930182,i dont understand why you dont want this one make me an offer or ill cry blok metaverse meta nfts nftcollector crypto art abstract nftcommunity eth digitalart doge shib safemoon
7,7,902694196150013952,1454461796761309186,veve will enter the metaverse playgroundveve omi token
8,8,1428821,1454461786531434511,the metaverse is already here in parts heres a dress i made in virtual on sale in the virtual world of
9,9,1441702148257955843,1454461785008795648,ilove elemongame partner corsairgamenft playtoearn topmvb binancesmartchainmetaverse


## Limpieza de datos para Polarización

In [85]:
df_polarizacion = pd.read_csv("metaverse.csv") 

print("Cantidad de filas antes de limpiar: ", len(df_polarizacion))

# ELimino tweets del author_id = 1441702148257955843, spam.
df_polarizacion = df_polarizacion.drop(df_polarizacion[df_polarizacion.author_id == 1441702148257955843].index)
# ELimino tweets del author_id = 1389675288255205378, spam.
df_polarizacion = df_polarizacion.drop(df_polarizacion[df_polarizacion.author_id == 1389675288255205378].index)
# ELimino tweets del author_id = 3343896580, spam.
df_polarizacion = df_polarizacion.drop(df_polarizacion[df_polarizacion.author_id == 3343896580].index)
# ELimino tweets del author_id = 1414202287681990667, spam.
df_polarizacion = df_polarizacion.drop(df_polarizacion[df_polarizacion.author_id == 1414202287681990667].index)

print("Cantidad de filas después de limpiar: ", len(df_polarizacion))

# removiendo tildes
df_polarizacion['text'] = df_polarizacion['text'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
# removiendo simbolo hashtag
df_polarizacion['text'] = df_polarizacion['text'].str.replace('#', '')
# removiendo menciones
df_polarizacion['text'] = df_polarizacion['text'].replace(r'(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', '', regex=True)
# removiendo caracteres numéricos
df_polarizacion['text'] = df_polarizacion['text'].replace(r'[0-9]+', '', regex=True)
# removiendo emojis
df_polarizacion = df_polarizacion.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))
# eliminando urls
df_polarizacion['text'] = df_polarizacion['text'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)
# removiendo saltos de linea, espacios en blanco y tabs
df_polarizacion['text'] = df_polarizacion['text'].str.replace('\n', '')
df_polarizacion['text'] = df_polarizacion['text'].str.replace('\t', '')
df_polarizacion['text'] = df_polarizacion['text'].str.replace(' {2,}', ' ', regex=True)
df_polarizacion['text'] = df_polarizacion['text'].str.strip()
# convirtiendo texto a minúscula
df_polarizacion['text'] = df_polarizacion['text'].str.lower()
# removiendo filas vacias
df_polarizacion.dropna()
df_polarizacion['text'].astype(bool)
df_polarizacion = df_polarizacion[df_polarizacion['text'].astype(bool)]

# Eliminando filas que no uso
#del df_polarizacion['author_id']
# del df_polarizacion['id']

# Imprimiendo resultado
df_polarizacion

Cantidad de filas antes de limpiar:  250
Cantidad de filas después de limpiar:  196


,Unnamed: 0,author_id,id,text
1,1,1448338535342231553,1454461853929639943,im definitely not underexposed i have s invested thousands in soul because is perfect for the metaverse phantasma is a layer blockchain with builtin smartnft technology perfect for nftgames and blockchaingaming
3,3,1440648481970950152,1454461830772899843,members will get sapce token randomlymetaverse presale gamefi nft playtoearn pe
4,4,44817501,1454461819372724238,and if you wanna know abt metaverse in a beginner friendly wayread this thread
6,6,1430501884192858112,1454461799210930182,i dont understand why you dont want this one make me an offer or ill cry blok metaverse meta nfts nftcollector crypto art abstract nftcommunity eth digitalart doge shib safemoon
7,7,902694196150013952,1454461796761309186,veve will enter the metaverse playgroundveve omi token
8,8,1428821,1454461786531434511,the metaverse is already here in parts heres a dress i made in virtual on sale in the virtual world of
10,10,1450099531144642564,1454461783457026054,they want to take away your substantial values through inflation taxation giving illusional digital stuff back to you with metaverse dont fall for that
11,11,461910412,1454461780181196802,i second that motion it is time btc eth grt link matic vet fear rad muon metaverse
12,12,1388502279238537219,1454461774737002496,io good project longterm vision im whiting airdrop but not selling daysim promising spat metaspatial metaverse nfts
13,13,1035791954,1454461756068139009,the metaverse cryptos are exploding since facebook announced meta theres like different cryptos in the space and i couldnt decide soooo i bought all of them not even kidding spray and pray see what sticks no missing out on any one opportunity lets go metaverse


## Polarización

In [86]:
import nltk
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# Instanciar Analizador
sentiment_analyzer = SentimentIntensityAnalyzer()

df_polarizacion["compound"] = ""
df_polarizacion["negative"] = ""
df_polarizacion["neutral"] = ""
df_polarizacion["positive"] = ""
df_polarizacion["result"] = ""
for index, row in df_polarizacion.iterrows():
    #Analizar cada review
    analisis = sentiment_analyzer.polarity_scores(row['text'])
    row["negative"] = analisis["neg"]
    row["neutral"] = analisis["neu"]
    row["positive"] = analisis["pos"]
    row["compound"] = analisis["compound"]
    # Evaluar que valores se considerarán positivo o negativo
    if analisis['compound'] ==  0.0: # Ajuste para que Compound == 0.0 no sea negativo
        row["result"] = "Neutral"
    elif analisis['compound'] >= 0.5 :
        row["result"] = "Positive"
    elif analisis['compound'] <  0.5:
        row["result"] = "Negative"
    else :
        row["result"] = "Neutral"
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', -1)

df_polarizacion

<ipython-input-86-5fc7d5befe60>:28: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Unnamed: 0,author_id,id,text,compound,negative,neutral,positive,result
1,1,1448338535342231553,1454461853929639943,im definitely not underexposed i have s invested thousands in soul because is perfect for the metaverse phantasma is a layer blockchain with builtin smartnft technology perfect for nftgames and blockchaingaming,0.8779,0.0,0.735,0.265,Positive
3,3,1440648481970950152,1454461830772899843,members will get sapce token randomlymetaverse presale gamefi nft playtoearn pe,0.0,0.0,1.0,0.0,Neutral
4,4,44817501,1454461819372724238,and if you wanna know abt metaverse in a beginner friendly wayread this thread,0.4939,0.0,0.802,0.198,Negative
6,6,1430501884192858112,1454461799210930182,i dont understand why you dont want this one make me an offer or ill cry blok metaverse meta nfts nftcollector crypto art abstract nftcommunity eth digitalart doge shib safemoon,-0.7288,0.209,0.791,0.0,Negative
7,7,902694196150013952,1454461796761309186,veve will enter the metaverse playgroundveve omi token,0.0,0.0,1.0,0.0,Neutral
8,8,1428821,1454461786531434511,the metaverse is already here in parts heres a dress i made in virtual on sale in the virtual world of,0.0,0.0,1.0,0.0,Neutral
10,10,1450099531144642564,1454461783457026054,they want to take away your substantial values through inflation taxation giving illusional digital stuff back to you with metaverse dont fall for that,0.7351,0.0,0.709,0.291,Positive
11,11,461910412,1454461780181196802,i second that motion it is time btc eth grt link matic vet fear rad muon metaverse,-0.4939,0.167,0.833,0.0,Negative
12,12,1388502279238537219,1454461774737002496,io good project longterm vision im whiting airdrop but not selling daysim promising spat metaspatial metaverse nfts,-0.1121,0.142,0.688,0.17,Negative
13,13,1035791954,1454461756068139009,the metaverse cryptos are exploding since facebook announced meta theres like different cryptos in the space and i couldnt decide soooo i bought all of them not even kidding spray and pray see what sticks no missing out on any one opportunity lets go metaverse,0.8016,0.026,0.788,0.187,Positive


In [79]:
from nltk.probability import FreqDist

polarizacion = df_polarizacion.explode('result')
# Obtener frecuencia de cada término
fdist = FreqDist(polarizacion['result'])
# Convertir a dataframe
df_fdist = pd.DataFrame.from_dict(fdist, orient='index')
df_fdist.columns = ['Resultados de polarización']
df_fdist.index.name = 'Polaridad'
df_fdist.sort_values(by=['Resultados de polarización'], inplace=True)
df_fdist

,Resultados de polarización
Polaridad,
Neutral,53
Negative,66
Positive,85


## Conclusión:

Se obtuvieron los tweets para el hashtag #Metaverse
Se realizo una limpieza antes de la Polarizació:
 - Se eliminaron tweets de dos autores que solo spameaban
 - Se removieron tildes (aunque no debería haber ya que los tweets son en Inglés)
 - Se removió el simbolo de hashtag #
 - Se removieron las menciones (no era necesario para aplicar polarización)
 - Se removieron caracteres numéricos
 - Se removieron emojis
 - Se removieron urls
 Se comparó la cantidad de filas obtenidas por api con las que quedaron luego de filtrar spam. 
 Finalmente se aplicó polarización
 - Ajusté el ancho máximo de la columna del tweet para ver todo el texto
 - Ajusté la cantidad de filas a mostrar para poder analizar 1 por 1
 - Se evalua si compound == 0.0. Actualmente esto hacia que la frase calificara como Negativa cuando en realidad era Neutra
 - Se ajustaron valores que se comparan contra Vader, ya que arrojaba falsos negativos.
 Se realiza una tabla de frecuencia con los resultados obtenidos

 Se concluye que al día de la fecha (30/10), los tweets con el hashtag #Metaverse en su mayoría son positivos.
 Contexto: El día 28/10/21, Mark Zuckerberg dió a conocer el nuevo nombre de Facebook. Este cambio de nombre está ligado a su próximo desarollo, Metaverse, una especie de mundo virtual en el que se pretende que podamos sociabilizar, jugar, comprar, trabajar, estudiar, entre otros. 
 Algunas tweet positivos mencionan:
 - Es un gran proyecto
 - Estoy muy emocionado por este equipo
 - Proyecto maravilloso y prometedor
 Algunas tweet negativos mencionan:
 - Podrías haberlo hecho mejor
 - Los problemas de privacidad están a punto de volverse 


 

